In [ ]:
from os import makedirs
from os.path import join, exists
from itertools import product
import numpy as np
import pandas as pd

from scipy import stats
import statsmodels.api as sm

from brainpipe.system import study
from mne.stats import fdr_correction, bonferroni_correction
from utils import rename_elecs, odors_su_score

In [ ]:
def su_gr_score_all(su,data):
    score_ = []
    for i,data_sel in enumerate(data):
        score_.extend([i+1]*data_sel.shape[-1])
    return score_

In [ ]:
"""
Correlate TPSim through learning as a function of trial order
>>>> SEMC and LEFC could be sliced differently (reducing the early group)
BY ELECTRODE and Plot summary (for all included electrodes)
"""
freq, meth = 'theta', '2gr' #'1_alpha', '2_beta','3_gamma', high_gamma
rois_sel = ['OFC_olf']
st = study('Olfacto')
###############################################################################
path_tps = join(st.path, 'feature/TPSim_3groups_Enc/TPS_btw_thgh_time_v=1_elecs=all/')
tps_form = join(path_tps, 'TPS_pears_learn_{}_btw_{}_{}.npz')
df_name = join(path_tps, '{}_ols_btw_{}_{}_{}.csv') #su, conds0, conds1, freq
###############################################################################
    
subjects = ['CHAF','LEFC','PIRJ','VACJ','SEMC','FERJ']
subjects_c, elecs_c, labels_c = np.array([]), np.array([]), np.array([])
tps_scores,channels_c, x_c, y_c, z_c = np.array([]),np.array([]), np.array([]), np.array([]), np.array([])
tps_sem, T_vals_c, p_vals_c = np.array([]), np.array([]),np.array([])
p_fdr_c, p_bf_c = np.array([]), np.array([])

for su in subjects:
    mat = np.load(tps_form.format(su,freq,meth),allow_pickle=True)

    #load all elec info,rename and select electrodes id
    labels, channels = mat['label'], mat['channel']
    x, y, z = mat['xyz'][:,0], mat['xyz'][:,1], mat['xyz'][:,2]
    labels_new = rename_elecs(labels,x,y,z)
    idx_sel = [i for i,lab in enumerate(labels_new) if lab in rois_sel]

    #selected data
    labels, channels = labels_new[idx_sel], channels[idx_sel]
    x, y, z = x[idx_sel], y[idx_sel], z[idx_sel]

    #load all data by trials
    tps_su, tps_sem_su = np.zeros((labels.shape[0],2)),np.zeros((labels.shape[0],2))
    data = []
    for n in range(len(mat.files)-3):
        data.append(mat['tps_'+str(n)][idx_sel,:])
    print('shape of tps data', [x.shape for x in data])
    
    #split data in 3 groups as a function of time and nb of trials
    if meth in ['3gr','btw_trials']:
        concat_ = np.concatenate(data,axis=-1)
        ntrials = np.sum([x.shape[-1] for x in data])
        nelecs = concat_.shape[0]
        print('concat', concat_.shape)
        if meth in ['3gr','btw_trials']:
            score_ = [0]*data[0].shape[-1]+[1]*data[1].shape[-1]+[2]*data[2].shape[-1]
        if meth == 'all':
            score_ = su_gr_score_all(su,data)
    if meth == '2gr':
        early, late = data[0],data[1]
        tps_su[:,0] += np.mean(early,axis=1)
        tps_sem_su[:,0] += stats.sem(early,axis=1)
        tps_su[:,1] += np.mean(late,axis=1)
        tps_sem_su[:,1] += stats.sem(late,axis=1)
        nelecs = early.shape[0]
        print(early.shape,late.shape)

    #compute stats Ttests-unpaired
    if meth in ['3gr','btw_trials']:
        T, unc_p = [], []
        for elec in range(nelecs):
            #Tval,pval = stats.kendalltau(all_tps[elec],all_scores)
            Y, X = np.array(concat_[elec]), sm.add_constant(np.array(score_))
            model_ols = sm.OLS(Y,X).fit()
            Tval, pval = np.round(model_ols.tvalues[1],3),model_ols.pvalues[1]
            T.append(Tval), unc_p.append(pval)
    if meth == '2gr':
        T, unc_p = stats.ttest_ind(early,late,axis=1,equal_var=False)
    _, p_fdr = fdr_correction(unc_p)
    _, p_bf = bonferroni_correction(unc_p)

    #fill all df data
    if early.shape[0] > 0:
        subjects_c = np.hstack((subjects_c,np.array([su]*nelecs))) if np.size(subjects_c) else np.array([su]*nelecs)
        elecs_c = np.hstack((elecs_c,np.arange(nelecs))) if np.size(elecs_c) else np.arange(nelecs)
        labels_c = np.hstack((labels_c,labels)) if np.size(labels_c) else labels
        channels_c = np.hstack((channels_c,channels)) if np.size(channels_c) else channels
        tps_scores = np.concatenate((tps_scores,tps_su)) if np.size(tps_scores) else tps_su
        tps_sem = np.concatenate((tps_sem,tps_sem_su)) if np.size(tps_sem) else tps_sem_su
        x_c = np.hstack((x_c,x)) if np.size(x_c) else x
        y_c = np.hstack((y_c,y)) if np.size(y_c) else y
        z_c = np.hstack((z_c,z)) if np.size(z_c) else z
        T_vals_c = np.hstack((T_vals_c,T)) if np.size(T_vals_c) else T
        p_vals_c = np.hstack((p_vals_c,unc_p)) if np.size(p_vals_c) else unc_p
        p_fdr_c = np.hstack((p_fdr_c,p_fdr)) if np.size(p_fdr_c) else p_fdr
        p_bf_c = np.hstack((p_bf_c,p_bf)) if np.size(p_bf_c) else p_bf

pvals_all_fdr = fdr_correction(p_vals_c)[1][:,np.newaxis]
data_all = np.concatenate((subjects_c[:,np.newaxis],labels_c[:,np.newaxis],
            channels_c[:,np.newaxis],x_c[:,np.newaxis],y_c[:,np.newaxis],
            z_c[:,np.newaxis],elecs_c[:,np.newaxis],tps_scores,tps_sem,T_vals_c[:,np.newaxis],
            p_vals_c[:,np.newaxis],p_fdr_c[:,np.newaxis],p_bf_c[:,np.newaxis],pvals_all_fdr),
            axis=1)
df = pd.DataFrame(data_all, columns=['subjects','labels','channels','x','y','z',
                        'elecs_num', 'tps_early', 'tps_late','tps_early_sem','tps_late_sem',
                                     'Tvals', 'unc_p','fdr_p', 'bonf_p','fdr_all'])
print(df.shape)
df.to_csv(df_name.format('All_subjects',freq,'learn'+meth,rois_sel[0]),index=False)  

In [ ]:
from collections import Counter

st = study('Olfacto')
olf_regions = ['Amg','pPirT','OFC_olf','Ins_olf']
freq, meth = 'theta', '2gr'
thrs = [0.05]
corrections = ['fdr_all']#['fdr_p','bonf_p']

##################################################################################
path_pow = join(st.path, 'feature/TPSim_3groups_Enc/TPS_btw_thgh_time_v=1_elecs=all/')
df_name = join(path_pow, '{}_ols_btw_{}_learn'+meth+'_aHC.csv') #su, conds0, conds1, freq
df_stat_save = join(path_pow, 'Bilan_{}_OLS_{}_{}_{}_{}.csv')
##################################################################################

df = pd.read_csv(df_name.format('All_subjects',freq))
print('Initial df shape', df.shape)

for th, corr in product(thrs,corrections):
    df_sel = df.loc[df[corr]<th]
    print(df_sel)
    #df_sel.to_csv(df_stat_save.format('All_subjects','theta',meth,roi,corr+str(th)))
    df_sel['sign'] = ['separation' if t > 0 else 'completion' for t in df_sel['Tvals']]
    print('stats at p < ',th, 'correction : ',corr, df_sel.shape)
    print(Counter(df['labels'].loc[df[corr]<th]))

    rois = np.unique(df_sel['labels'])
    for roi in rois:
        if roi in ['aHC','OFC_olf','IFG']:
            df_roi = df_sel.loc[df_sel['labels']==roi]
            #print(df_roi)
            df_inc = df_roi.loc[df_roi['sign']=='completion'].groupby(['subjects']).count()
            df_dec = df_roi.loc[df_roi['sign']=='separation'].groupby(['subjects']).count()

            if (df_inc.shape[0] >= 1) or (df_inc.shape[0] >=2 and roi in olf_regions):
                print(roi, 'NB of subjects with completion',df_inc.shape[0],' subjects')
                df_plot = df_roi.loc[df_roi['sign']=='completion']
                print('#electrodes in total >>> ',df_plot.shape[0])
                df_plot.to_csv(df_stat_save.format('All_subjects','theta',meth,roi,corr+str(th)))
                print(df_plot)
                
            if (df_dec.shape[0] >= 1) or (df_dec.shape[0] >=2 and roi in olf_regions):
                print(roi, 'NB of subjects with separation',df_dec.shape[0],' subjects')
                df_plot = df_roi.loc[df_roi['sign']=='separation']
                print('#electrodes in total >>> ',df_plot.shape[0])
                df_plot.to_csv(df_stat_save.format('All_subjects','theta',meth,roi,corr+str(th)))
                print(df_plot)


### Plot Early Late effects
Individual results bar plots

In [ ]:
from brainpipe.system import study
import matplotlib.pyplot as plt
from os.path import join
import pandas as pd
import numpy as np

st = study('Olfacto')
PATH = join(st.path, 'feature/TPSim_3groups_{}/TPS_btw_thgh_time_v=1_elecs=all/')
file_name = join(PATH, 'All_subjects_ols_btw_theta_learn2gr_{}.csv')
SAVE_PATH = join(PATH, 'Ind_elecs_{}_{}_avg_subj_2gr_not_sig.png')
meth, exp, freq, rois = 'btw', 'Enc', 'theta', ['aHC','OFC_olf']
features = ['tps_early','tps_late']

for roi in rois:
    df = pd.read_csv(file_name.format(exp,roi))
    df_m = df.groupby(['subjects']).agg(['mean','sem'])
    nelecs = df_m.shape[0]

    fig, axs = plt.subplots(2,4, figsize=(15, 6), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()

    for i in range(nelecs):
        df_sel = df_m.iloc[i,:].unstack().T
        #print(df_sel)

        su =  df_m.index[i]
        data = df_sel[features].iloc[0,:].values
#         yerr = df_sd.iloc[i,:][features].values
        yerr = df_sel[features].iloc[1,:].values
        #print(data,yerr)

        xticks, w = np.arange((len(features))), 0.8
        axs[i].set_xticks(xticks,features)
        axs[i].bar(xticks,data,yerr=yerr,color='blue')
        axs[i].set_title('{} in {}'.format(su,roi))
    
    plt.setp(axs, ylim=(0,1)) if roi == 'aHC' else plt.setp(axs, ylim=(0,1.1))
    plt.savefig(SAVE_PATH.format(exp,'tps',roi))
    plt.savefig(SAVE_PATH.format(exp,'tps',roi).replace('.png','.pdf'))
    plt.clf()
    plt.close()
    

In [ ]:
from scipy.stats import pearsonr, spearmanr

def tpsim_by_cond(pow1, stat='pearson',average=True):
    """
    Compute tpsim within one condition for all combinations
    Parameters
    ----------
    pow1 : array 
        Must be of shape (npts x ntrials)
    stat : string
        The stat correlation method to use. 'pearson' or 'spearman'
    Returns
    -------
    tpsim : array
    """
    
    corr = pearsonr if stat == 'pearson' else spearmanr
    print(pow1.shape)
    sim_trials = np.array([])
    for t0, t1 in combinations(np.arange(pow1.shape[-1]),2):
        R, _ = corr(pow1[:,t0],pow1[:,t1])
        sim_trials = np.vstack((sim_trials,1-R)) if np.size(sim_trials) else np.array([1-R])
    print('sim_trials shape', sim_trials.shape)
    if np.size(sim_trials) == 1:
        sim_trials = np.array([[sim_trials]])
    else:
        sim_trials = sim_trials.swapaxes(0,1)
    if average == True:
        sim_trials = np.mean(sim_trials)
    return sim_trials

def early_late_pow(pow_data):
    thr = int(pow_data.shape[-1]/2)
    if thr == 0:
        early = np.array([])
        late = np.array([])
    else:
        early = pow_data[...,:thr]
        late = pow_data[...,thr:]
    return early, late

def tpsim_btw_2odors(pow_o1,pow_o2,stat='pearson',average=True):
    """
    Compute tpsim between 2 odors 
    Parameters
    ----------
    pow_o1, pow_o2 : array (npts x ntrials)
    stat : string
            The stat correlation method to use. 'pearson' or 'spearman'
    Returns
    -------
    tpsim : array (Average TPSim between 2 ODORS)
    """
    
    corr = stats.pearsonr if stat == 'pearson' else stats.spearmanr
    sim_trials = np.array([])
    for t0, t1 in product(range(pow_o1.shape[-1]),range(pow_o2.shape[-1])):
        R, _ = corr(pow_o1[:,t0],pow_o2[:,t1])
        sim_trials = np.vstack((sim_trials,1-R)) if np.size(sim_trials) else np.array([1-R])
    if average == True:
        sim_trials = np.mean(sim_trials)
    return sim_trials


In [ ]:
from itertools import combinations_with_replacement, combinations
"""
Compute TPSim BTW odors By ODOR EARLY and LATE ALLTOGETHER
>>> in order to compute dissimilarity matrices EARLY and LATE
"""
###############################################################################
st = study('Olfacto')
path_pow = join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_odor_all_elecsFT/')
pow_path = join(path_pow, '{}_odor_{}_bipo_all_{}_6freqs.npz')
path2save = join(st.path, 'feature/TPSim_3groups_{}/similarity_matrix_thgh_time_btw_v=1_elecs=all/')
tps_save = join(path2save, 'TPS_{}_{}_{}_EL_btw_odors_mean.npz')
###############################################################################
exp, stat = 'Enc', 'pearson' #Ret, Enc
shortcut = ['E','R'] if exp == 'Enc_Ret' else [exp[0]]
if not exists(path2save.format(exp)):
    makedirs(path2save.format(exp))
###############################################################################
freqs, f = ['theta'], 1
odors_su = {'CHAF': [1,2,4,5,3,8,7,9], #low than high
            'LEFC': [15,2,1,16,14,3,4,17],
            'PIRJ': [1,9,5,4,6,7,18], #missing odor 15
            'VACJ': [11,14,12,10,15,17,16,13],
            'SEMC': [7,10,11,12,13,5,8,9],
            'FERJ': [7,2,16,17,12,1,5,13]}
    
for su,freq in product(odors_su,freqs):
    list_od = []
    for od in odors_su[su]:
        pow_ = np.load(pow_path.format(su,str(od),exp[0],freq))['xpow']
        if pow_.shape[-1] >= 2:
            list_od.append(od)
    odors_EL = [str(o)+'_E' for o in list_od] + [str(o)+'_L' for o in list_od]
    ncomb = len([c for c in combinations(odors_EL,2)]) #different odors only
    od_sel = odors_EL[0].split('_')[0]
    mat_ = np.load(pow_path.format(su,od_sel,exp[0],freq),allow_pickle=True)
    chans, labels, xyz = mat_['channels'], mat_['labels'], mat_['xyz']
    nelecs = mat_['xpow'][f,...].shape[0]
    print(su,freq,list_od,'nb comb',ncomb,'nelecs',nelecs)
    
    tps = np.zeros((nelecs,ncomb))
    list_comb = np.array([])
    for i,comb in enumerate(combinations(odors_EL,2)):
        print('o1',comb[0],'o2',comb[1])
        mat_o1 = np.load(pow_path.format(su,comb[0].split('_')[0],exp[0],freq),allow_pickle=True)
        pow_o1 = mat_o1['xpow'][f,:,17:47,:] #nelecs,npts,ntrials
        mat_o2 = np.load(pow_path.format(su,comb[1].split('_')[0],exp[0],freq),allow_pickle=True)
        pow_o2 = mat_o2['xpow'][f,:,17:47,:] #nelecs,npts,ntrials

        o1_E, o1_L = early_late_pow(pow_o1)
        o2_E, o2_L = early_late_pow(pow_o2)
        print(o1_E.shape,o1_L.shape,o2_E.shape,o2_L.shape,)
        
        list_comb = np.vstack((list_comb,comb)) if np.size(list_comb) else comb
        
        for elec in range(nelecs):
            o1_sel = o1_E if comb[0].split('_')[1] == 'E' else o1_L
            o2_sel = o2_E if comb[1].split('_')[1] == 'E' else o2_L
            tps[elec,i] += tpsim_btw_2odors(o1_sel[elec],o2_sel[elec],average=True)
            
    np.savez(tps_save.format(exp,stat,su,freq),tps_EL=tps,comb=list_comb,channels=chans,
                labels=labels, xyz=xyz)

In [ ]:
from itertools import combinations_with_replacement
"""
Compute TPSim BTW odors By ODOR FOR EARLY or LATE separately
>>> in order to compute dissimilarity matrices EARLY and LATE
"""
###############################################################################
st = study('Olfacto')
path_pow = join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_odor_all_elecsFT/')
pow_path = join(path_pow, '{}_odor_{}_bipo_all_{}_6freqs.npz')
path2save = join(st.path, 'feature/TPSim_3groups_{}/similarity_matrix_thgh_time_btw_v=1_elecs=all/')
tps_save = join(path2save, 'TPS_{}_{}_{}_btw_odors_mean.npz')
###############################################################################
exp, stat = 'Enc', 'pearson' #Ret, Enc
shortcut = ['E','R'] if exp == 'Enc_Ret' else [exp[0]]
if not exists(path2save.format(exp)):
    makedirs(path2save.format(exp))
###############################################################################
freqs, f = ['theta'], 1
odors_su = {'CHAF': [1,2,4,5,3,8,7,9], #low than high
            'LEFC': [15,2,1,16,14,3,4,17],
            'PIRJ': [1,9,5,4,6,7,18], #missing odor 15
            'VACJ': [11,14,12,10,15,17,16,13],
            'SEMC': [7,10,11,12,13,5,8,9],
            'FERJ': [7,2,16,17,12,1,5,13]}
    
for su,freq in product(odors_su,freqs):
    list_od = []
    for od in odors_su[su]:
        pow_ = np.load(pow_path.format(su,str(od),exp[0],freq))['xpow']
        if pow_.shape[-1] >= 2:
            list_od.append(od)
    ncomb = len([c for c in combinations_with_replacement(list_od,2)])
    od_sel = list_od[0]
    mat_ = np.load(pow_path.format(su,str(od_sel),exp[0],freq),allow_pickle=True)
    chans, labels, xyz = mat_['channels'], mat_['labels'], mat_['xyz']
    nelecs = mat_['xpow'][f,...].shape[0]
    print(su,freq,list_od,'nb comb',ncomb,'nelecs',nelecs)
    tpsE, tpsL = np.zeros((nelecs,ncomb)), np.zeros((nelecs,ncomb))
    
    list_comb = np.array([])
    for i,comb in enumerate(combinations_with_replacement(list_od,2)):
        print('o1',comb[0],'o2',comb[1])
        mat_o1 = np.load(pow_path.format(su,comb[0],exp[0],freq),allow_pickle=True)
        pow_o1 = mat_o1['xpow'][f,:,17:47,:] #nelecs,npts,ntrials
        mat_o2 = np.load(pow_path.format(su,comb[1],exp[0],freq),allow_pickle=True)
        pow_o2 = mat_o2['xpow'][f,:,17:47,:] #nelecs,npts,ntrials
        o1_E, o1_L = early_late_pow(pow_o1)
        o2_E, o2_L = early_late_pow(pow_o2)
        print(o1_E.shape,o1_L.shape,o2_E.shape,o2_L.shape,)
        
        list_comb = np.vstack((list_comb,comb)) if np.size(list_comb) else comb
        
        for elec in range(nelecs):
            if comb[0] != comb[1]: #different odors
                tpsE[elec,i] += tpsim_btw_2odors(o1_E[elec],o2_E[elec],average=True)
                tpsL[elec,i] +=tpsim_btw_2odors(o1_L[elec],o2_L[elec],average=True)
            else: #if same odor o1 = o2
                if o1_E.shape[-1] > 1: #compute similiarities across repetitions of the same odor
                    tpsE[elec,i] += tpsim_by_cond(o1_E[elec],average=True)
                if o1_L.shape[-1] > 1:
                    tpsL[elec,i] += tpsim_by_cond(o1_L[elec],average=True)
            
    np.savez(tps_save.format(exp,stat,su,freq),tpsE=tpsE, tpsL=tpsL,comb=list_comb,channels=chans,
                labels=labels, xyz=xyz)

### Plot distance matrix for Early and Late

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib import cm
from matplotlib.patches import Patch
from sklearn.manifold import MDS

from brainpipe.system import study
import numpy as np
from os.path import join
import random
from utils import  odor_groups_3wgth
"""
Plot TPSim matrices (RDM) for all odors 
"""
###############################################################################
st = study('Olfacto')
path_df = join(st.path, 'feature/TPSim_3groups_Enc/TPS_btw_thgh_time_v=1_elecs=all/')
df_name = join(path_df, 'Bilan_All_subjects_OLS_theta_2gr_{}_fdr_all0.05.csv')
path_npz = join(st.path,'feature/TPSim_3groups_Enc/similarity_matrix_thgh_time_btw_v=1_elecs=all')
npz_name = join(path_npz, 'TPS_pearson_{}_theta_EL_btw_odors_mean.npz')
savename = join(path_npz, 'Plot_distance_{}_{}_{}_EL.png')
###############################################################################
rois = ['aHC','OFC_olf']
conds = ['tps_EL']

for roi in rois:
    df = pd.read_csv(df_name.format(roi))
    subjects = np.unique(df['subjects'])
    for su in subjects:
        df_su = df.loc[df['subjects']==su]
        chans_df = df_su['channels'].values
        
        mat = np.load(npz_name.format(su),allow_pickle=True)
        print(su,'shape', mat[conds[0]].shape)
        idx_elecs = [i for i,chan in enumerate(mat['channels']) if chan in chans_df]
        combs = mat['comb']
        print(mat['labels'][idx_elecs], mat['channels'][idx_elecs])
        n_od = len(np.unique(combs))
        print('n_od',n_od)
        idx = [combs[0,0]]+[o for o in combs[:n_od-1,1]]
        #idx = [o for o in combs[:n_od,1]]
        
        tps = mat[conds[0]][idx_elecs] if len(idx_elecs) == 1 else \
                                    np.mean(mat[conds[0]][idx_elecs],axis=0)
        tri = np.zeros((n_od, n_od))
        #tri[np.triu_indices(n_od)] = tps #diagonal of 0
        #tri[np.tril_indices(n_od)] = tri.T[np.tril_indices(n_od)]
        
        tri[np.triu_indices(n_od,1)] = tps #diagonal of 0
        tri[np.tril_indices(n_od, -1)] = tri.T[np.tril_indices(n_od, -1)]
        
        model = MDS(n_components=2, dissimilarity='precomputed') #random_state=1
        out = model.fit_transform(tri)

        fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(8,3))
        colors = ['aqua']*int(n_od/2)+['darkorange']*int(n_od/2)
        markers = 'o'
        x_centroid_E = sum(out[:int(n_od/2),0])/int(n_od/2) 
        y_centroid_E = sum(out[:int(n_od/2),0])/int(n_od/2)
        x_centroid_L = sum(out[int(n_od/2):,0])/int(n_od/2)
        y_centroid_L = sum(out[int(n_od/2):,0])/int(n_od/2)
        
        for i, txt in enumerate(idx):
            ax1.scatter(out[i,0], out[i,1], c=colors[i], marker=markers,s=6)
            ax1.annotate(txt, (out[i,0], out[i,1]))

        for i, txt in enumerate(idx):
            ax2.scatter(out[i,0], out[i,1], c=colors[i], marker=markers,s=6)
        ax2.scatter(x_centroid_E, y_centroid_E, s=6,c='aqua', marker='*')
        ax2.scatter(x_centroid_L, y_centroid_L, s=6,c='darkorange', marker='*')
        
        #subplot imshow matrix and mask upper triangle
        mask =  np.tri(tri.shape[0], k=0) #mask upper triangle
        for row,col in product(range(tri.shape[0]), range(tri.shape[1])):
            if row < (tri.shape[0]/2):
                if col > ((tri.shape[0]-1)/2):
                    mask[row,col] = 1 #mask early vs late distances
        
        A = np.ma.array(tri, mask=mask) # mask out the lower triangle
        cmap = cm.get_cmap('viridis', 30)
        cmap.set_bad('w') # default value is 'k'
        cax = ax3.imshow(A,vmin=0,vmax=1.8,interpolation="nearest", cmap=cmap,aspect='auto')
        ax3.set_xticks(np.arange(n_od))
        ax3.set_yticks(np.arange(n_od))
        ax3.set_xticklabels(idx,fontsize=11,rotation=60)
        ax3.set_yticklabels(idx,fontsize=11,rotation=20)

        asp = np.abs(np.diff(ax1.get_xlim())[0] / np.diff(ax1.get_ylim())[0])
        ax1.set_aspect(asp)
        ax2.set_aspect(asp)
        
        plt.colorbar(cax)
        plt.subplots_adjust(top=0.92)
        title = 'Distance btw odors in {} {} {}'.format(su,'early_late',roi)
        fig.suptitle(title, fontsize=12)

        plt.savefig(savename.format(su,'early_late',roi))
        plt.savefig(savename.format(su,'early_late',roi).replace('.png','.pdf'))
        plt.clf()
        plt.close()


In [ ]:
mask =  np.tri(tri.shape[0], k=0)
print(tri.shape)
print(mask)

print(mask)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib import cm
from matplotlib.patches import Patch
from sklearn.manifold import MDS

from brainpipe.system import study
import numpy as np
from os.path import join
import random
from utils import  odor_groups_3wgth
"""
Plot TPSim matrices (RDM) for all odors 
"""
###############################################################################
st = study('Olfacto')
path_df = join(st.path, 'feature/TPSim_3groups_Enc/TPS_btw_thgh_time_v=1_elecs=all/')
df_name = join(path_df, 'Bilan_All_subjects_OLS_theta_2gr_{}_fdr_all0.05.csv')
path_npz = join(st.path,'feature/TPSim_3groups_Enc/similarity_matrix_thgh_time_btw_v=1_elecs=all')
npz_name = join(path_npz, 'TPS_pearson_{}_theta_btw_odors_mean.npz')
savename = join(path_npz, 'Plot_distance_{}_{}_{}_2D.png')
###############################################################################
rois = ['aHC','OFC_olf']
conds = ['tpsE', 'tpsL']

for roi in rois:
    df = pd.read_csv(df_name.format(roi))
    subjects = np.unique(df['subjects'])
    for su in subjects:
        df_su = df.loc[df['subjects']==su]
        idx_elecs = df_su['elecs_num']
        
        mat = np.load(npz_name.format(su),allow_pickle=True)
        print(su,'shape', mat['tpsE'].shape)
        combs = mat['comb']
        n_od = len(np.unique(combs))
        print('n_od',n_od)
        idx = list([o for o in combs[:n_od,1]])
        
        tpsE = mat[conds[0]][idx_elecs] if len(idx_elecs) == 1 else np.mean(mat[conds[0]][idx_elecs],axis=0)
        triE = np.zeros((n_od, n_od))
        triE[np.triu_indices(n_od)] = tpsE
        triE[np.tril_indices(n_od)] = triE.T[np.tril_indices(n_od)]
        
        tpsL = mat[conds[1]][idx_elecs] if len(idx_elecs) == 1 else np.mean(mat[conds[1]][idx_elecs],axis=0)
        triL = np.zeros((n_od, n_od))
        triL[np.triu_indices(n_od)] = tpsL
        triL[np.tril_indices(n_od)] = triL.T[np.tril_indices(n_od)]

        model = MDS(n_components=2, dissimilarity='precomputed', random_state=1)
        out0 = model.fit_transform(triE)
        out1 = model.fit_transform(triL)
        out_all = np.concatenate((out0,out1),axis=0)
        idx_all = [str(i)+'_E' for i in idx]+[str(i)+'_L' for i in idx]

        fig, (ax1) = plt.subplots(1,1,figsize=(4,3))
        colors = ['purple']*n_od+['orange']*n_od
        markers = 'o'

        for i, txt in enumerate(idx_all):
            print(i,txt)
            ax1.scatter(out_all[i,0], out_all[i,1], c=colors[i], marker=markers)
            ax1.annotate('O'+str(txt), (out_all[i,0], out_all[i,1]))
        ax1.set_xlabel('component 1')
        ax1.set_ylabel('component 2')
        #ax1.axis('equal')
        
        plt.subplots_adjust(top=0.92)
        title = 'Distance btw odors in {} {} {})'.format(su,cond,roi)
        fig.suptitle(title, fontsize=12)

        plt.savefig(savename.format(su,cond,roi))
        plt.savefig(savename.format(su,cond,roi).replace('.png','.pdf'))
        plt.clf()
        plt.close()


### Select electrodes from memory effect 

In [ ]:
path_elecs = join(st.path, 'feature/TPSim_3groups_Enc/LinReg_stats_theta_v=1_elecs=all/')
path_df_learn = join(st.path, 'feature/TPSim_3groups_Enc/TPS_btw_thgh_time_v=1_elecs=all/')
meth = '2gr'
files = st.search('Bilan_All_subjects',folder = path_elecs)

for fi in files:
    df = pd.read_csv(path_elecs+fi)
    roi_st = fi.split('_')[-4]+'_'+fi.split('_')[-3]+'_'+fi.split('_')[-2][:-4]
    subjects = np.unique(df['subjects'].values)
    dict_elecs = {}
    for su in subjects:
        df_sel = df.loc[df['subjects']==su]
        dict_elecs[su] = list(df_sel['channels'].values)
    
    df2 = pd.read_csv(path_df_learn+'All_subjects_ols_btw_theta_learn'+meth+'.csv')
    for i,su in enumerate(dict_elecs):
        if i == 0:
            df_elecs = df2.loc[(df2['subjects']==su) & (df2.channels.isin(dict_elecs[su]))]
        else:
            df_el = df2.loc[(df2['subjects']==su) & (df2.channels.isin(dict_elecs[su]))]
            df_elecs = df_elecs.append(df_el)
    df_elecs.to_csv(path_df_learn+'Bilan_elecs_mem_{}_{}.csv'.format(meth,roi_st),index=False)
    print(df_elecs)

### Check for Early Late effect all mem groups together

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import sem, ttest_rel, ttest_ind
import statsmodels.api as sm
import statsmodels.formula.api as smf

"""
ANOVA à mesures répétées, Mixed ANOVA ou Ttest paired
"""
freq = 'theta' #'1_alpha', '2_beta','3_gamma', high_gamma
st = study('Olfacto')
###############################################################################
path_tps = join(st.path, 'feature/TPSim_3groups_Enc/TPS_btw_thgh_time_v=1_elecs=all/')
###############################################################################
steps = ['early','late']

files = st.search('Bilan_All_', folder=path_tps)
pvals = []
for fi in files:
    df = pd.read_csv(path_tps+fi)
    #df = df.groupby(['subjects']).mean()
    n_rows = df.shape[0]
    subj_ = np.concatenate([df.index.values]*len(steps))[:,np.newaxis]
    tps = df[['tps_early','tps_late']].values.T.flatten()[:,np.newaxis]
    steps_ = np.concatenate([[step]*n_rows for step in steps])[:,np.newaxis]
    df_stat = pd.DataFrame(data=np.concatenate((subj_,tps,steps_),axis=1),
                  columns=['subjs','tps','steps'])
    df_stat['tps'] = df_stat['tps'].astype(float)
    print(df_stat)
    0/0
    #md = smf.mixedlm("tps ~ steps", df_stat, groups=df_stat["subjs"])
    #mdf = md.fit()
    #print(mdf.summary())
    Tval, pval = ttest_rel(df['tps_early'].values,df['tps_late'].values)
    print(Tval, pval)
    #anova = pg.rm_anova(dv='tps',within=['steps'],
    #              subject='subjs', data=df_stat)
    #pvals.append(anova['p-unc'][0])
    #print(anova)
    cols0 = [col for col in df_gr[conds_df] if col.endswith('0')]
    cols1 = [col for col in df_gr[conds_df] if col.endswith('1')]
    means0, means1 = df_gr[cols0].values.mean(), df_gr[cols1].values.mean()
    sem0, sem1 = sem(np.ravel(df_gr[cols0].values)), sem(np.ravel(df_gr[cols1].values))
    ax = plt.bar(np.arange(2),np.array([means0,means1]), yerr=np.array([sem0,sem1]))
    print(fi)    
    #ax = df[conds_df].mean().plot.bar(yerr=df[conds_df].sem())
    plt.savefig(path_tps+fi.replace('.csv','.png').replace('Bilan','Fig'))
    plt.savefig(path_tps+fi.replace('.csv','.pdf').replace('Bilan','Fig'))
    plt.clf()
    plt.close()


### Compute sig elecs by mem

In [ ]:
from itertools import product
"""
Compute df with TPSim through learning as a function of subsequent memory effect
"""
freq, conds = 'theta', ['low','mid','high'] #'1_alpha', '2_beta','3_gamma', high_gamma
rois_sel = ['aHC','MFG','ACC','IFG','Amg','pPirT','PHG','Ins_olf','OFC_olf','SFG']
time_steps = ['early','late']
st = study('Olfacto')
###############################################################################
path_tps = join(st.path, 'feature/TPSim_3groups_Enc/TPS_btw_thgh_time_v=1_elecs=all/')
tps_form = join(path_tps, 'TPS_pears_learn_{}_btw_{}_{}_2gr.npz')
df_name = join(path_tps, '{}_ols_btw_{}_{}.csv') #su, conds0, conds1, freq
###############################################################################
    
subjects = ['CHAF','LEFC','PIRJ','VACJ','SEMC','FERJ']
subjects_c, elecs_c, labels_c = np.array([]), np.array([]), np.array([])
tps_scores, channels_c, x_c, y_c, z_c = np.array([]),np.array([]), np.array([]), np.array([]), np.array([])

for su in subjects:
    mat = np.load(tps_form.format(su,freq,conds[0]),allow_pickle=True)

    #load all elec info,rename and select electrodes id
    labels, channels = mat['label'], mat['channel']
    x, y, z = mat['xyz'][:,0], mat['xyz'][:,1], mat['xyz'][:,2]
    labels_new = rename_elecs(labels,x,y,z)
    idx_sel = [i for i,lab in enumerate(labels_new) if lab in rois_sel]
    nelecs = len(idx_sel)

    #selected data
    labels, channels = labels_new[idx_sel], channels[idx_sel]
    x, y, z = x[idx_sel], y[idx_sel], z[idx_sel]

    #load all data by trials
    tps_su = np.zeros((labels.shape[0],6))
    i = 0
    for cond in conds:
        data = np.load(tps_form.format(su,freq,cond))
        for n in range(len(time_steps)):
            tps_su[:,i] += np.mean(data['tps_'+str(n)][idx_sel,:],axis=1)
            i += 1
    #fill all df data
    subjects_c = np.hstack((subjects_c,np.array([su]*nelecs))) if np.size(subjects_c) else np.array([su]*nelecs)
    elecs_c = np.hstack((elecs_c,np.arange(nelecs))) if np.size(elecs_c) else np.arange(nelecs)
    labels_c = np.hstack((labels_c,labels)) if np.size(labels_c) else labels
    channels_c = np.hstack((channels_c,channels)) if np.size(channels_c) else channels
    tps_scores = np.concatenate((tps_scores,tps_su)) if np.size(tps_scores) else tps_su
    x_c = np.hstack((x_c,x)) if np.size(x_c) else x
    y_c = np.hstack((y_c,y)) if np.size(y_c) else y
    z_c = np.hstack((z_c,z)) if np.size(z_c) else z

data = np.concatenate((subjects_c[:,np.newaxis],labels_c[:,np.newaxis],
            channels_c[:,np.newaxis],x_c[:,np.newaxis],y_c[:,np.newaxis],
            z_c[:,np.newaxis],elecs_c[:,np.newaxis],tps_scores),axis=1)
df = pd.DataFrame(data, columns=['subjects','labels','channels','x','y','z',
                'elecs_num', 'low_0','low_1', 'mid_0','mid_1',
                'high_0','high_1'])
print(df.shape)
df.to_csv(df_name.format('All_subjects',freq,'learn_conds2gr'),index=False)  

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import sem
import statsmodels.api as sm
import statsmodels.formula.api as smf
"""
ANOVA à mesures répétées pour les électrodes sig en mem LinReg
"""
freq, conds = 'theta', ['low','mid','high'] #'1_alpha', '2_beta','3_gamma', high_gamma
rois_sel = ['aHC','MFG','ACC','IFG','Amg','pPirT','PHG','Ins_olf','OFC_olf','SFG']
st = study('Olfacto')
###############################################################################
path_tps = join(st.path, 'feature/TPSim_3groups_Enc/TPS_btw_thgh_time_v=1_elecs=all/')
tps_form = join(path_tps, 'TPS_pears_learn_{}_btw_{}_{}_2gr.npz')
df_name = join(path_tps, '{}_ols_btw_{}_{}.csv') #su, conds0, conds1, freq
###############################################################################
conds_df = ['low_0','low_1','mid_0','mid_1','high_0','high_1']
conds = ['low','mid','high']
steps = ['early','late']

files = st.search('Bilan_elecs_mem_conds', folder=path_tps)
pvals = []
for fi in files:
    df = pd.read_csv(path_tps+fi)
    #df_gr = df.groupby(['channels']).mean()
    df_gr = df
    n_su = df_gr.shape[0]
    subj_ = np.concatenate([df_gr['subjects'].values]*len(conds_df))[:,np.newaxis]
    tps = df_gr[conds_df].values.T.flatten()[:,np.newaxis]
    conds_ = np.concatenate([[cond]*(n_su*2) for cond in conds])[:,np.newaxis]
    steps_ = np.concatenate([[step]*n_su for step in steps]*len(conds))[:,np.newaxis]
    df_stat = pd.DataFrame(data=np.concatenate((subj_,tps,conds_,steps_),axis=1),
                  columns=['subjs','tps','conds','steps'])
    df_stat['tps'] = df_stat['tps'].astype(float)
    print(df_stat.loc[df_stat['subjs']=='PIRJ'].groupby(['steps','conds']).mean())
    df_stat = df_stat.groupby(['subjs','steps']).mean()
    df_stat = df_stat.reset_index()
    print(df_stat)
    #md = smf.mixedlm("tps ~ steps", df_stat, groups=df_stat["subjs"])
    #mdf = md.fit()
    #print(mdf.summary())
    anova = pg.rm_anova(dv='tps',within=['steps'],
                  subject='subjs', data=df_stat)
    pvals.append(anova['p-unc'][0])
    print(anova)
    cols0 = [col for col in df_gr[conds_df] if col.endswith('0')]
    cols1 = [col for col in df_gr[conds_df] if col.endswith('1')]
    means0, means1 = df_gr[cols0].values.mean(), df_gr[cols1].values.mean()
    sem0, sem1 = sem(np.ravel(df_gr[cols0].values)), sem(np.ravel(df_gr[cols1].values))
    ax = plt.bar(np.arange(2),np.array([means0,means1]), yerr=np.array([sem0,sem1]))
    print(fi)    
    #ax = df[conds_df].mean().plot.bar(yerr=df[conds_df].sem())
    plt.savefig(path_tps+fi.replace('.csv','.png').replace('Bilan','Fig'))
    plt.savefig(path_tps+fi.replace('.csv','.pdf').replace('Bilan','Fig'))
    plt.clf()
    plt.close()


In [15]:
import matplotlib.pyplot as plt
from scipy.stats import sem, ttest_ind
import numpy as np
from utils import d_su_gr_fam_pl, rename_elecs
from brainpipe.system import study
from os.path import join
from itertools import product
from mne.stats import bonferroni_correction, fdr_correction
import pandas as pd
"""
TPD Early/Late as a function of Memory Richness, Pleasantness or Familiarity
"""
freq = 'theta' #'1_alpha', '2_beta','3_gamma', high_gamma
steps = ['early','late']
feats = {'fam':['low_f','high_f'],
        'pleas':['neu_pl', 'ext_pl']}
rois_sel = ['OFC_olf']

st = study('Olfacto')
###############################################################################
path_tps = join(st.path, 'feature/TPSim_3groups_Enc/by_odor_btw_thgh_time_v=1_elecs=all/')
tps_form = join(path_tps, 'TPS_pears_learn_{}_btw_{}_{}_2gr.npz') #su, od, freq
save_path = join(st.path, 'feature/TPSim_3groups_Enc/Ttests_early_late_fam_pl_TPD/')
df_name = join(save_path, '{}_Ttests_{}_{}.csv') 
###############################################################################

for step, feat in product(steps,feats):
    subjects_c, labels_c = np.array([]), np.array([])
    channels_c, x_c, y_c, z_c = np.array([]), np.array([]), np.array([]), np.array([])
    tps0_c, tps1_c, sem0_c, sem1_c = np.array([]), np.array([]), np.array([]), np.array([])
    T_vals, p_vals = np.array([]), np.array([])
    
    for su in d_su_gr_fam_pl:
        conds = []
        for cond in feats[feat]:
            print(cond)
            btw_tps = np.array([])
            for od in d_su_gr_fam_pl[su][cond]:
                mat = np.load(tps_form.format(su,od,freq),allow_pickle=True)
                data = mat['tps_0'] if step == 'early' else mat['tps_1']
                btw_tps = np.concatenate((btw_tps,data),axis=1) if np.size(btw_tps) else data
            conds.append(btw_tps)
        
        #select electrodes
        labels, channels = mat['label'], mat['channel']
        x, y, z = mat['xyz'][:,0], mat['xyz'][:,1], mat['xyz'][:,2]
        labels_new = rename_elecs(labels,x,y,z)
        idx_sel = [i for i,lab in enumerate(labels_new) if lab in rois_sel]
        nelecs = len(idx_sel)
        labels, channels = labels_new[idx_sel], channels[idx_sel]
        x, y, z = x[idx_sel], y[idx_sel], z[idx_sel]
        sel_data = [1 - x[idx_sel,:] for x in conds]
        print('after selection for',su,feat,step,[x.shape for x in sel_data])
        
        if sel_data[0].shape[0] > 0:
            #compute stats (unpaired Ttests)
            Tvals_, unc_p = ttest_ind(sel_data[0],sel_data[1],axis=1)
            tps0, tps1 = np.mean(sel_data[0],axis=1), np.mean(sel_data[1],axis=1)
            sem0, sem1 = sem(sel_data[0],axis=1), sem(sel_data[1],axis=1)

            #concatenate all data in order to create the df
            subjects_c = np.hstack((subjects_c,np.array([su]*nelecs))) if np.size(subjects_c) else np.array([su]*nelecs)
            labels_c = np.hstack((labels_c,labels)) if np.size(labels_c) else labels
            channels_c = np.hstack((channels_c,channels)) if np.size(channels_c) else channels
            x_c = np.hstack((x_c,x)) if np.size(x_c) else x
            y_c = np.hstack((y_c,y)) if np.size(y_c) else y
            z_c = np.hstack((z_c,z)) if np.size(z_c) else z

            #add stats and descriptive values (sem, mean)
            tps0_c = np.hstack((tps0_c,tps0)) if np.size(tps0_c) else tps0
            tps1_c = np.hstack((tps1_c,tps1)) if np.size(tps1_c) else tps1
            sem0_c = np.hstack((sem0_c,sem0)) if np.size(sem0_c) else sem0
            sem1_c = np.hstack((sem1_c,sem1)) if np.size(sem1_c) else sem1
            T_vals = np.hstack((T_vals,Tvals_)) if np.size(T_vals) else Tvals_
            p_vals = np.hstack((p_vals,unc_p)) if np.size(p_vals) else unc_p
    _, p_fdr_ = fdr_correction(p_vals)
    _, p_bf_ = bonferroni_correction(p_vals)

    data = np.concatenate((subjects_c[:,np.newaxis],labels_c[:,np.newaxis],
                channels_c[:,np.newaxis],x_c[:,np.newaxis],y_c[:,np.newaxis],
                z_c[:,np.newaxis],tps0_c[:,np.newaxis], tps1_c[:,np.newaxis], 
                sem0_c[:,np.newaxis],sem1_c[:,np.newaxis],
                T_vals[:,np.newaxis],p_vals[:,np.newaxis],
                p_fdr_[:,np.newaxis],p_bf_[:,np.newaxis]), axis=1)
    df = pd.DataFrame(data, columns=['subjects','labels','channels','x','y','z',
                'tps_'+feats[feat][0], 'tps_'+feats[feat][1], 
                'tps_sem_'+feats[feat][0], 'tps_sem_'+feats[feat][1], 
                'Tvals','unc_p','fdr_p','bonf_p'])
    print(df.shape)
    df.to_csv(df_name.format('All_subjects',feat,step+'_'+freq),index=False)

-> Olfacto loaded
low_f
high_f
after selection for CHAF fam early [(3, 22), (3, 16)]
low_f
high_f
after selection for LEFC fam early [(5, 190), (5, 366)]
low_f
high_f
after selection for PIRJ fam early [(4, 84), (4, 60)]
low_f
high_f
after selection for VACJ fam early [(4, 38), (4, 32)]
low_f
high_f
after selection for SEMC fam early [(3, 172), (3, 98)]
low_f
high_f
after selection for FERJ fam early [(0, 68), (0, 72)]
(19, 14)
neu_pl
ext_pl
after selection for CHAF pleas early [(3, 16), (3, 22)]
neu_pl
ext_pl
after selection for LEFC pleas early [(5, 286), (5, 270)]
neu_pl
ext_pl
after selection for PIRJ pleas early [(4, 60), (4, 84)]
neu_pl
ext_pl
after selection for VACJ pleas early [(4, 32), (4, 38)]
neu_pl
ext_pl
after selection for SEMC pleas early [(3, 96), (3, 174)]
neu_pl
ext_pl
after selection for FERJ pleas early [(0, 82), (0, 58)]
(19, 14)
low_f
high_f
after selection for CHAF fam late [(3, 60), (3, 36)]
low_f
high_f
after selection for LEFC fam late [(5, 214), (5, 484)]
lo

In [20]:
import matplotlib.pyplot as plt
from scipy.stats import sem, ttest_ind
import numpy as np
from utils import d_su_gr_fam_pl, rename_elecs
from brainpipe.system import study
from os.path import join
from itertools import product
from mne.stats import bonferroni_correction, fdr_correction
import pandas as pd
"""
Delta TPD Early/Late as a function of Memory Richness, Pleasantness or Familiarity
"""
freq = 'theta' #'1_alpha', '2_beta','3_gamma', high_gamma
feats = {'fam':['low_f','high_f'],
        'pleas':['neu_pl', 'ext_pl']}
rois_sel = ['OFC_olf']

st = study('Olfacto')
###############################################################################
path_tps = join(st.path, 'feature/TPSim_3groups_Enc/by_odor_btw_thgh_time_v=1_elecs=all/')
tps_form = join(path_tps, 'TPS_pears_learn_{}_btw_{}_{}_2gr.npz') #su, od, freq
save_path = join(st.path, 'feature/TPSim_3groups_Enc/Ttests_early_late_fam_pl_TPD/')
df_name = join(save_path, '{}_Ttests_{}_{}.csv') 
###############################################################################

for feat in feats:
    subjects_c, labels_c = np.array([]), np.array([])
    channels_c, x_c, y_c, z_c = np.array([]), np.array([]), np.array([]), np.array([])
    tps0_c, tps1_c, sem0_c, sem1_c = np.array([]), np.array([]), np.array([]), np.array([])
    T_vals, p_vals = np.array([]), np.array([])
    
    for su in d_su_gr_fam_pl:
        conds0, conds1 = [], []
        for cond in feats[feat]:
            print(cond)
            btw_tps0, btw_tps1 = np.array([]), np.array([])
            for od in d_su_gr_fam_pl[su][cond]:
                mat = np.load(tps_form.format(su,od,freq),allow_pickle=True)
                data0, data1 = mat['tps_0'], mat['tps_1']
                btw_tps0 = np.concatenate((btw_tps0,data0),axis=1) if np.size(btw_tps0) else data0
                btw_tps1 = np.concatenate((btw_tps1,data1),axis=1) if np.size(btw_tps1) else data1
            conds0.append(btw_tps0), conds1.append(btw_tps1)
        
        #select electrodes
        labels, channels = mat['label'], mat['channel']
        x, y, z = mat['xyz'][:,0], mat['xyz'][:,1], mat['xyz'][:,2]
        labels_new = rename_elecs(labels,x,y,z)
        idx_sel = [i for i,lab in enumerate(labels_new) if lab in rois_sel]
        nelecs = len(idx_sel)
        labels, channels = labels_new[idx_sel], channels[idx_sel]
        x, y, z = x[idx_sel], y[idx_sel], z[idx_sel]
        sel_data0 = [1 - x[idx_sel,:] for x in conds0]
        sel_data1 = [1 - x[idx_sel,:] for x in conds1]
        print('after selection for',su,feat,step,[x.shape for x in sel_data1])
        
        #create the diff of TPD
        diff_data_l, diff_data_h = np.array([]), np.array([])
        for elec in range(nelecs):
            d0_l, d1_l = sel_data0[0][elec], sel_data1[0][elec]
            d0_h, d1_h = sel_data0[1][elec], sel_data1[1][elec]
            diff_l = np.array([y-x for x,y in product(d0_l,d1_l)])[np.newaxis]
            diff_h = np.array([y-x for x,y in product(d0_h,d1_h)])[np.newaxis]
            diff_data_l = np.concatenate((diff_data_l,diff_l),axis=0) if np.size(diff_data_l) else diff_l
            diff_data_h = np.concatenate((diff_data_h,diff_h),axis=0) if np.size(diff_data_h) else diff_h
        diff_all = [diff_data_l, diff_data_h]
        
        if diff_all[0].shape[0] > 0:
            #compute stats (unpaired Ttests)
            Tvals_, unc_p = ttest_ind(diff_all[0],diff_all[1],axis=1)
            tps0, tps1 = np.mean(diff_all[0],axis=1), np.mean(diff_all[1],axis=1)
            sem0, sem1 = sem(diff_all[0],axis=1), sem(diff_all[1],axis=1)

            #concatenate all data in order to create the df
            subjects_c = np.hstack((subjects_c,np.array([su]*nelecs))) if np.size(subjects_c) else np.array([su]*nelecs)
            labels_c = np.hstack((labels_c,labels)) if np.size(labels_c) else labels
            channels_c = np.hstack((channels_c,channels)) if np.size(channels_c) else channels
            x_c = np.hstack((x_c,x)) if np.size(x_c) else x
            y_c = np.hstack((y_c,y)) if np.size(y_c) else y
            z_c = np.hstack((z_c,z)) if np.size(z_c) else z

            #add stats and descriptive values (sem, mean)
            tps0_c = np.hstack((tps0_c,tps0)) if np.size(tps0_c) else tps0
            tps1_c = np.hstack((tps1_c,tps1)) if np.size(tps1_c) else tps1
            sem0_c = np.hstack((sem0_c,sem0)) if np.size(sem0_c) else sem0
            sem1_c = np.hstack((sem1_c,sem1)) if np.size(sem1_c) else sem1
            T_vals = np.hstack((T_vals,Tvals_)) if np.size(T_vals) else Tvals_
            p_vals = np.hstack((p_vals,unc_p)) if np.size(p_vals) else unc_p
    _, p_fdr_ = fdr_correction(p_vals)
    _, p_bf_ = bonferroni_correction(p_vals)

    data = np.concatenate((subjects_c[:,np.newaxis],labels_c[:,np.newaxis],
                channels_c[:,np.newaxis],x_c[:,np.newaxis],y_c[:,np.newaxis],
                z_c[:,np.newaxis],tps0_c[:,np.newaxis], tps1_c[:,np.newaxis], 
                sem0_c[:,np.newaxis],sem1_c[:,np.newaxis],
                T_vals[:,np.newaxis],p_vals[:,np.newaxis],
                p_fdr_[:,np.newaxis],p_bf_[:,np.newaxis]), axis=1)
    df = pd.DataFrame(data, columns=['subjects','labels','channels','x','y','z',
                'tps_'+feats[feat][0], 'tps_'+feats[feat][1], 
                'tps_sem_'+feats[feat][0], 'tps_sem_'+feats[feat][1], 
                'Tvals','unc_p','fdr_p','bonf_p'])
    print(df.shape)
    df.to_csv(df_name.format('All_subjects',feat,'deltaTPD_'+freq),index=False)

-> Olfacto loaded
low_f
high_f
after selection for CHAF fam late [(3, 60), (3, 36)]
low_f
high_f
after selection for LEFC fam late [(5, 214), (5, 484)]
low_f
high_f
after selection for PIRJ fam late [(4, 108), (4, 82)]
low_f
high_f
after selection for VACJ fam late [(4, 78), (4, 68)]
low_f
high_f
after selection for SEMC fam late [(3, 254), (3, 160)]
low_f
high_f
after selection for FERJ fam late [(0, 128), (0, 146)]
(19, 14)
neu_pl
ext_pl
after selection for CHAF pleas late [(3, 42), (3, 54)]
neu_pl
ext_pl
after selection for LEFC pleas late [(5, 350), (5, 348)]
neu_pl
ext_pl
after selection for PIRJ pleas late [(4, 82), (4, 108)]
neu_pl
ext_pl
after selection for VACJ pleas late [(4, 68), (4, 78)]
neu_pl
ext_pl
after selection for SEMC pleas late [(3, 177), (3, 237)]
neu_pl
ext_pl
after selection for FERJ pleas late [(0, 161), (0, 113)]
(19, 14)
